### TODO
- [ ] Create table of contents mimicking [this](https://sebastianraschka.com/Articles/2014_ipython_internal_links.html)
- [ ] Export utils functions, like play_audio and tada, to outer file

In [5]:
%pycat ../../../personal_utils.py

### Functions and data useful for any kind of project



from IPython.display import HTML, Audio

def play_audio(sound_file: str):
    """ Plays outloud sound_file """
    display(Audio(url=sound_file, autoplay=True))
    
    
def tada():
    tada_path = "../../../tada.mp3"
    play_audio(tada_path)
    
def tada2():
    tada_path2 = "./tada.mp3"
    play_audio(tada_path2)


In [40]:
from IPython.display import HTML, Audio

def play_audio(sound_file: str):
    """ Plays outloud sound_file """
    display(Audio(url=sound_file, autoplay=True))
    
    
def tada():
    tada_path = "../../../tada.mp3"
    play_audio(tada_path)

# Table of Contents:

<!-- [Building the Spam Filter](Building the Spam Filter) <br> -->

<a href='#Tokenization'>4) Tokenization and Context</a>

# How to Build a Spam Filter using Bayes Theorem
---
We want to build a Spam filter that classifies e-mails in two groups: **fraud** and **no fraud** sales.
Our e-mails can have features like "has_promotional_codes" or "has_giftcards" that we could use to make better judgments about if an e-mail contains a fraudulent sale or not.
But how do we feed this information to a spam classifier?

We can convert this information to proportions (i.e., probabilities) and use Bayes theorem to train a classifier.

A **Naive Bayes Classifier** is a supervised and *probabilistic* learning model. It works well for problems where:
* Data for which the inputs are independent from one another
* Data where the probability of any attribute is greater than zero, always

### Conidtional Probabilities

\begin{equation}
P(A|B) = \frac{P(A \cap B)}{P(B)}
\end{equation}

The **intersection function $ \cap $** (informally called *cap* due to its shape) can also be thought of as the Boolean operator **AND** applied on sets. In Python it looks like:
```python
a = [1, 2, 3]; b = [1, 4, 5]

set(a) & set(b)  # => {1}
set(a).intersection(set(b)) == set(a) & set(b)  # => True
```

The **union function $\cup$** (informally called *cup* due to its shape) can be thought of as the **OR** operator:
```python
set(a) | set(b)  # => {1, 2, 3, 4, 5}
set(a).union(set(b)) == set(a) | set(b)  # => True
```
We can see the idea of conditional probability in Python like this:
```python
A, B = set(a), set(b)
total_cases = len(A) + len(B)

p_A_cap_B = len(A & B) / total_cases  # => 0.16, probability of intersection happening
p_B = len(B) / total_cases  # => 0.5, probability of B happening

# So, according to the conditional probability formula above
p_A_given_B = p_A_cap_B / p_B  # => 0.333

# which always satisfies the following inequality:
p_B > p_A_given_B > p_A_cap_B  # => True
```

### Inverse Conditional Probability (*aka* Bayes Theorem)

But what if we want to know the opposite case, that is, what if we want to know the probability of B, given A?.
In that case, we can reason backwards and recalling that $ A \cap B = B \cap A $:

From the conditional probability formula above, we know that
$$ P(A \cap B) = P(A|B)P(B) $$

So, starting again with the conditional probability and substituting the expression for $ P(B \cap A) $ we have
$$ P(B|A) = \frac{P(A \cap B)}{P(A)} = \frac{P(A|B)P(B)}{P(A)} $$

which is known as **Bayes Theorem**. We state it again for simplicity:

$$ P(B|A) = \frac{P(A|B)P(B)}{P(A)} $$

### Naive Bayesian Classifier

#### The Chain Rule

Using the joint probability, the previous result transforms into the chain rule.<br>
Joint probabilities are the probability that all the events will happen at the same time. The generic form is:
$$
P(A_1, A_2, ..., A_n) = P(A_1)P(A_2|A_1)P(A_3 | A_1, A_2)···P(A_n | A_1, A_2, ..., A_{n-1})
$$


##### Navieté in Bayesian Reasoning

Sometimes, when we strictly apply Bayes Theorem, we can run into trouble because we need to use values of probabilities that we cannot compute. For example, $ P(Promos | Giftcards, Fraud) $ can be very hard, or even impossible, to get.
But we still need to get going. We solve this problem by doing a rather big assumption (the *Navieté assumption*) to help us proceed in cases where some information is too difficult or impossible to get. The **Navieté** assumption consists in **considering only the individual effect that each event (Giftcard, Promos, ...) has in the event we want to predict**, in our case, fraud sales e-mails.

$$ 
\text{what we want to know} = P(Fraud | Giftcard, Promos) = \frac{P(Giftcard, Promos, Fraud)}{P(Giftcard, Promos)}
$$

$ P(Giftcard, Promos) $ is easy to obtain, and thanks to the Navieté assumption (no interdependence of things like *having promo codes* and *having giftcards* (which really is unrealistic) we can simplify the numerator to an expression that we can really work with, given our data. So our numerator becomes:

$$ 
P(Giftcard, Promos, Fraud) = P(Fraud)P(Giftcard|Fraud)P(Promos|Fraud)
$$

and our final model being:

$$
P(Fraud | Giftcard, Promos) = \frac{P(Fraud)P(Giftcard | Fraud)P(Promos | Fraud)}{P(Giftcard, Promos)}
$$



But what if something like $ P(Promos | Fraud) = 0 $? That could happen due to new information, or not considering enough data. In that case, since probabilites here are just divisions of counts, we use what's called a **pseudocount**, which is just the real count of a class plus 1. In that way, we ensure that every computed probability will be always greater than zero.

# Building the Spam Filter



Here we have the coding design for this exercise:

<img src="images/coding_design.jpeg" style="width: 190px;">

Each `Email` object takes an `.eml` text file that then tokenizes into something that the `SpamTrainer` can utilize. <br>
When testing, we will focus on the tradeoff between false positives and false negatives, since in this scenario, a **false positive** (filtering out an e-mail as SPAM when it is not) **could be very harmful for a business**. Thus, we will try to **minimize the false positive rate**.

#### Data source

* **CSDMC2010 SPAM corpus** <br>
This data set has 4,327 total messages, of which 2,949 are ham and 1,378 are spam.

## Intro to Test-Driven Development
----

Let's first define the class that will appropriately parse the e-mails in the `.eml` text files. But since we will use **TDD**, before we actually implement the class, we must first build the class(es) that will put to the test each desired functionality for that class. In this manner, after we have developed the class, we have a judge (the test) that objectively tells us whether or not the class does what we want them to do. <br>
The steps are the following:

##### 1 ) Write the test class for the desired class
If you want to build a class *ClassX*, first build a class *TestClassX* inside which you use *ClassX* as if it were already implemented, according to the functionalities (methods and attributes) that you envision for it.

Represent what you want the class to do inside the test class. Any method that starts with `test_` will be treated as a test to run. You can write auxiliary methods that do not follow this name pattern. <br>
Also, each test class needs a method called `setUp`, whose job is to "set it all up" for the proper generation of the actual result and the expected result, so that they can be compared.

```python
class TestClassX(unittest.TestCase):
    def setUp(self, ...):
        # read/prepare data as ClassX would expected to be
        
    def test_functionality_1(self):
        classX_instance = ClassX()
        result = classX_instance.method_func1()  # method_func1 does not exist yet as this point
        expected = function_that_does_the_right_thing()  # or a hardcoded "true answer"
        self.assertEqual(result, expected)
```
##### 2) Implement the method that will do the functionality 
```python
class ClassX(object):
    def method_func1(self):
        result_to_be_compared_with_expected_in_test = # do stuff
        return result_to_be_compared_with_expected_in_test
```

##### 3) Run the test and see whether it passes.
Recommended to store or the test classes in the same file for later automation  # TODO: How exactly?

In [42]:
import unittest
import io
import re

#### Test # 1: `EmailObject`

We want `EmailObject` to just parse the **subject** and the **body** of each email. So we'll have a method for each. <br>

1) We begin by creating a class test targeted at the functionality of reading **plain text** e-mails:

In [43]:
class TestPlaintextEmailObject(unittest.TestCase):
    """ Assumes there is a class EmailObject imported """
    
    CRLF = "\r\n\r\n"  # carriage return and line feed.
                       # Separates headers in our .eml files
    
    def setUp(self):
        """
        Loads example file against which the unit tests will be performed, and instantiates the 
        Email class on this file for later tests.
        """
        self.plain_file = './tests/fixtures/plain.eml'
        with open(self.plain_file, 'rb') as plaintext:
            self.text = plaintext.read().decode('utf-8')  # text needed for the expected part of tests
            plaintext.seek(0)  # point to start, so EmailObject can read it
            # instantiate the to-be-tested class
            self.plain_email = EmailObject(plaintext)  # expects binary files, not strings
        
    def test_parse_plain_body(self):
        """ Check if body of email is extracted properly """
        body_expected = self.CRLF.join(self.text.split(self.CRLF)[1:])  # split on CRLF, take all except first one
        body_actual = self.plain_email.body()  # behaviour of body method, not implemented yet
        # compare method result with expected result
        self.asserEqual(body_actual, body_expected)
        
    def test_parses_the_subject(self):
        """ Check if subject of email is extracted properly """
        subject_expected = re.search(pattern="Subject: (.*)", string=self.text).group(1)
        subject_actual = self.plain_email.subject()
        # compare
        self.asserEqual(subject_actual, subject_expected)

Now that we have a test class, let's build the *first iteration* of its associated class that will make all the test pass.

**NOTE**: Instead of relying purely on *Regular Expressions* for this text processing tasks, we will use Python's standard library `email` for this problem. Of course, this means our custom methods for `EmailObject` will depend on Python's `email` library functions.

In [44]:
import email

class EmailObject(object):
    """
    Parses e-mails from .eml files.
      ^
     /|\  1st Iteration
    /_·_\ 
    """
    def __init__(self, email_binary_file, category=None):
        self.email_binary_file = email_binary_file
        self.category = category
        self.mail = email.message_from_binary_file(self.email_binary_file)  # TODO: returns what type?
        
    def subject(self):
        return self.mail.get('Subject')
        
    def body(self):
        return self.mail.get_payload(decode=True)  # TODO: type returned?

2) Now we create a class test targeted at the functionality of reading **HTML** e-mails. For that, we need to capture only the `inner_text`.

In [19]:
from bs4 import BeautifulSoup

class TestHTMLEmail(unittest.TestCase):
    """
    Assumes existence of an EmailObject.
    Tests whether EmailOject properly reads HTML files
    """
    CRLF = "\n\n"
    def setUp(self):
        # Prepare attributes to do the expected-vs-actual comparisons
        with open('./tests/fixtures/html.eml', mode='rb') as html_file:
            self.html = html_file.read().decode('utf-8')  # needed for the expected result part
            html_file.seek(0)
            # save object output to be tested below
            self.html_email = EmailObject(html_file)  # accepts binary text files
            
    def test_parses_and_stores_inner_text_html(self):
        body_temp = CRLF.join(self.html.split(CRLF)[1:])  # take all except first one
        body_expected = BeautifulSoup(body_temp).text  # add 'html.parser' as 2nd argument if goes wrong
        body_actual = self.html_email.body()  # tests method
        self.assertEqual(body_actual, body_expected)
        
    def test_stores_subject(self):
        subject_expected = re.search(pattern="Subject: (.*)", string=self.html).group(1)
        subject_actual = self.html_email.subject()  # tests method
        self.assertEqual(subject_actual, subject_expected)

Since we also need to detect the `content_type`, and not only the `inner_text`, we'll add a new feature to the body method in our `EmailObject` class that does that:

In [27]:
class EmailObject(object):
    """
    Parses e-mails from .eml files.
      ^
     /|\  2nd Iteration ==> Final version
    /_·_\ 
    """
    default_text_return = ''
    
    def __init__(self, email_binary_file, category=None):
        self.email_binary_file = email_binary_file
        self.category = category
        self.mail = email.message_from_binary_file(self.email_binary_file)  # TODO: returns what type?
        
    def subject(self):
        return self.mail.get('Subject')
        
    def body(self):
        body_temp = self.mail.get_payload(decode=True)  # TODO: type returned?
        content_type = self.mail.get_content_type()
        # knowing content_type allows us to generalize the previous body method to HTML files
        if content_type == 'text/html':
            return BeautifulSoup(body_temp).text
        elif content_type == 'text/plain':
            return body_temp
        else:
            return self.default_text_return

Now we have a working email parser, but we still have to deal with *tokenization*, or what to extract from the body and subject.

### Tokenization and Context
Because we are building a **Naive** Bayesian Classifier, we are *assuming* that each individual token is contributing independently to the *spamminess* of the e-mail.

The goal of our tokenizer will be to extract words into a stream in order to keep a **low memory profile**.

As always, the process is the following:

1) Make a conceptual or functional desing of what the class will do <br>
2) Make a class test that will test those functionalities <br>
3) Implement the actual class that will succeed the tests
4) Run the tests

In [38]:
class TestTokenizer(unittest.TestCase):
    """
    Checks that tokenization is properly done
    """
    def setUp(self):
        self.string = "quick brown fox"
        
    def test_downcase(self):
        expected = self.string.split()
        actual = Tokenizer.tokenize(list(map(str.upper, expected)))
        self.assertEqual(actual, expected)
        
    def test_ngram(self):
        """
        Tests result of ngram for n = 2
        """
        expected = [
            [u'\u0000', "quick"],
            ["quick", "brown"],
            ["brown", "fox"],  # TODO: Is this comma here right?
        ]
        actual = Tokenizer.ngram(self.string, 2)
        self.assertEqual(actual, expected)

In [49]:
class Tokenizer(object):
    """
    Tokenizes words of email
    """ 
    NULL = u'\u0000'
    
    @staticmethod  # decorator used for self-contained utility functions used only in current class
    def tokenize(string):
        return re.findall(pattern='\w+', string=string.lower())
        
    @staticmethod
    def ngram(string, n):
        """ 
        Doc  # TODO
        """
        tokens = Tokenizer.tokenize(string)
        
        ngrams = []
        
        for i in range(len(tokens)):
            shift = (i - n) + 1
            padding = max(-shift, 0)
            first_idx = max(shift, 0)
            last_idx = first_idx + n - padding
            
            ngrams.append(Tokenizer.pad(tokens[first_idx:last_idx], padding))
        
        return ngrams
        
    @staticmethod
    def pad(tokens, padding):
        """
        # TODO: Doc, and explain briefly what this does.
        """
        padded_tokens = []
        
        for i in range(padding):
            padded_tokens.append(Tokenizer.NULL)
            
        return padded_tokens + tokens

Now that we have a way of parsing and tokenizing e-mails, we can move on to build the **Bayesian** portion of our spam filter: the **`SpamTrainer`**.

It will do 3 things: 
#### - Storing training data
 - Storing a set of all categories
 - Storing a *unique* word count for each category
 - Storing the totals for each category
 
#### - Building a Bayesian Classifier
<!--  - asdasd -->

#### - Error minimization through cross-validation
<!-- - asdasd -->

---
Storing a set of all category names (ham, spam, scram):

In [78]:
class TestSpamTrainer(unittest.TestCase):
    """
    Tests whether .....  # TODO
    """
    def setUp(self):
        self.training = [  # List[List[key, value]]
            ['spam', './tests/fixtures/plain.eml'],
            ['ham', './tests/fixtures/small.eml'],
            ['scram', './tests/fixtures/plain.eml']
        ]
        self.trainer = SpamTrainer(self.training)
        with open('./tests/fixtures/plain.eml', 'rb') as file:
            self.email = EmailObject(file)
        
    def test_multiple_categories(self):
        """ Capture all category names from the e-mails """
        expected = set(k for k, _ in self.training)
        actual = self.trainer.categories()
        self.assertEqual(actual, expected)
        
    def test_counts_all_at_zero(self):
        expected_count = 0
        for cat in ['_all', 'spam', 'ham', 'scram']:
            actual_count = self.trainer.total_for(cat)
            self.assertEqual(expected_count, actual_count)

In [ ]:
from collections import defaultdict

class SpamTrainer(object):
    """
    Docstring
    """
    def __init__(self, training_files):
        self.categories = set()
        
        for category, file in training_files:
            self.categories.add(category)
        
        self.totals = defaultdict(float)
        self.training = {categ: defaultdict(float)
                         for categ in self.categories}
        self.to_train = training_files
        
    def total_for(self, category):
        """ Doc """
        return self.totals[category]
        
    def categories(self):
        """ Doc """
        
    def method1(self):
        """ Doc """
        
    def method1(self):
        """ Doc """
        
    

Try methods here before putting them in the class

In [60]:
class EmailObject:
    """ Parses incoming email messages """
    
    CRLF = "\r\n\r\n"  # carriage return and line feed. Separates headers

    def __init__(self, infile, category=None):
        """ Initializes an Email with its filepath, label and data (i.e., text) """
        self.infile = infile
        self.category = category
        self.mail = email.message_from_binary_file(self.infile)
    
    def subject(self) -> str:
        """ Extracts the subject of the email """
        return self.mail.get("Subject")
        
    def body(self):
        """ Extracts the body of the email """
        payload = self.mail.get_payload(decode=True)
        content_type = self.mail.get_content_type()
        if content_type == 'text/html':
            return BeautifulSoup(payload).text
        elif content_type == 'text/plain':
            return payload
        else:
            return ''

<a id='Tokenization'></a>

### Tokenization and Context

<img src="images/Tokenization.png" style="width: 400px;">

In [74]:
def test_parse_plain_body(self=None):
        """
        Doc
        """
        

## [References](./references_ch4.md)